## You can skip this part I am just organizing my files

In [ ]:
import os
import shutil

# Modify the paths
image_dir = 'C:/projects/SLT_project_MP_images/valid/images'
label_dir = 'C:/projects/SLT_project_MP_images/valid/labels'
dest_dir = 'C:/projects/SLT_project_MP_images/valid/'

#Create folders for each letter (A-Z)
for i in range(26):
    letter = chr(65 + i)  # 65 is ASCII for 'A'
    folder_path = os.path.join(dest_dir, letter)
    os.makedirs(folder_path, exist_ok=True)

# Organize images based on labels
def organize_images():
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)

        # Read label file to get class ID
        with open(label_path, 'r') as file:
            first_line = file.readline()
            class_id = int(first_line.split()[0])  # Extract class ID (first value)

        # Determine corresponding folder (A = 0, B = 1, ..., Z = 25)
        target_folder = chr(65 + class_id)
        target_folder_path = os.path.join(dest_dir, target_folder)

        # Find the corresponding image by matching filename
        image_name = label_file.rsplit('.', 1)[0] + '.jpg'
        image_path = os.path.join(image_dir, image_name)

        # Move image and label to target folder
        if os.path.exists(image_path):
            shutil.move(image_path, os.path.join(target_folder_path, image_name))
            

        print(f'Moved {image_name} to {target_folder}')




In [8]:
organize_images()

Moved A-10-_jpg.rf.f9c5cabf980d84ff9bfda9eb57117116.jpg to A
Moved A-11-_jpg.rf.b569d73a231e3bb1b3064e43f0e4229e.jpg to A
Moved A-3-_jpg.rf.77bd6739aeb6c95251f8a61235200758.jpg to A
Moved A_2_jpg.rf.6f6084fd9062bd4e92fed24d6c10c997.jpg to A
Moved B-10-_jpg.rf.593ee959e0c62506ca69d83f0ffaf8fa.jpg to B
Moved B-14-_jpg.rf.4b938c55a9799a728d06f907b65159e2.jpg to B
Moved B-8-_jpg.rf.02eea81a635dc058c5fec3e1487ca498.jpg to B
Moved B_2_jpg.rf.5d57d98ddee81df179247afeed393f97.jpg to B
Moved C-13-_jpg.rf.d8f9dd7760094a3725887a8d5f350621.jpg to C
Moved C-4-_jpg.rf.d31b78dffee431d73e822bd7bc6b3413.jpg to C
Moved C_3_jpg.rf.e973f89d8ccabdf43014cfba9f9e1f14.jpg to C
Moved C_4_jpg.rf.130f9108a34e5dca77e69722fda872e3.jpg to C
Moved D-12-_jpg.rf.1d1bd7a79f6cf5ca23d0214a0a405890.jpg to D
Moved D-5-_jpg.rf.baf7ab9e5b98e9dd7000cb60b6ef0333.jpg to D
Moved D-9-_jpg.rf.230e9adcb4ceb1b695877192efc5b65d.jpg to D
Moved D_5_jpg.rf.218b1880f46ffae1c248c96399ba80c7.jpg to D
Moved E-14-_jpg.rf.f9e21421c29fb2cbf250

## Build the model

In [1]:
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
from IPython.display import clear_output

class LivePlotCallback(Callback):
    def __init__(self):
        self.losses = []
        self.val_losses = []
        self.accuracies = []
        self.val_accuracies = []

    def on_epoch_end(self, epoch, logs=None):
        # Append training and validation loss/accuracy
        self.losses.append(logs['loss'])
        self.val_losses.append(logs['val_loss'])
        self.accuracies.append(logs['accuracy'])
        self.val_accuracies.append(logs['val_accuracy'])
        
        # Clear the current output and plot
        clear_output(wait=True)
        plt.figure(figsize=(12, 5))

        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(self.losses, label='Training Loss')
        plt.plot(self.val_losses, label='Validation Loss')
        plt.title('Loss Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        # Plot accuracy
        plt.subplot(1, 2, 2)
        plt.plot(self.accuracies, label='Training Accuracy')
        plt.plot(self.val_accuracies, label='Validation Accuracy')
        plt.title('Accuracy Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.show()





In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# Initialize MediaPipe Hands module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
# I want only to detect one hand but you can modify as you want
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Extract hand landmarks function
def extract_hand_landmarks(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmark_list = []
            for lm in hand_landmarks.landmark:
                landmark_list.append(lm.x)
                landmark_list.append(lm.y)
            return np.array(landmark_list).flatten()
    
    # Return array of zeros if no hands are detected
    return np.zeros(42)  # 21 landmarks * 2 (x, y)

# Build hand gesture classifier model

def build_hand_gesture_classifier_with_cnn():
    model = keras.Sequential([
        # Reshape input for CNN (42 -> 21 landmarks x 2 coordinates)
        layers.Reshape((21, 2, 1), input_shape=(42,)),

        # Convolutional layers
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.3),

        # Flatten for dense layers
        layers.Flatten(),

        # Fully connected layers
        layers.Dense(256, activation='relu'),
        

        layers.Dense(128, activation='relu'),
        

        # Output layer
        layers.Dense(26, activation='softmax')  # 26 classes (A-Z)
    ])

    return model

def build_hand_gesture_classifier_lstm():
    # You can add lstm layers but I removed them because they didn't make a difference
    model = keras.Sequential([
        layers.Input(shape=(42,)),  # 21 landmarks * 2 (x, y)
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        # layers.Dense(64, activation='relu'), # when this layer was removed model worked a little bit better
        layers.Dense(26, activation='softmax')  # 26 classes (A-Z)
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Preprocess image and extract landmarks
def process_image(image_path):
    image = cv2.imread(image_path)
    landmarks = extract_hand_landmarks(image)
    return landmarks  # (42,)

# Prepare dataset
def prepare_dataset(data_dir, label_to_index):
    images = []
    labels = []

    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        if os.path.isdir(class_path):
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                landmarks = process_image(image_path)
                images.append(landmarks)
                labels.append(label_to_index[class_folder])  # Map class_folder to integer label
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# Train the model
def train_model(model, train_dir, val_dir, label_to_index, index_to_label, epochs=200, batch_size=16):
    # Prepare datasets
    x_train, y_train = prepare_dataset(train_dir, label_to_index)
    x_val, y_val = prepare_dataset(val_dir, label_to_index)

    early_stopping = EarlyStopping(
        monitor='val_loss',   # Metric to monitor (you can also use 'val_accuracy')
        patience=40,          # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore weights of the best epoch
        verbose=1             # Print logs
    )
    # Train the model
    live_plot = LivePlotCallback()
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[live_plot, early_stopping]
    )
    # Evaluate the model on validation set
    y_pred = np.argmax(model.predict(x_val), axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(y_val, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(index_to_label.values()))
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Confusion Matrix')
    plt.show()

    return history

# Perform prediction
def predict_gesture(model, image_path, label_to_index, index_to_label):
    input_data = np.expand_dims(process_image(image_path), axis=0)
    prediction = model.predict(input_data)
    predicted_class = np.argmax(prediction)
    class_label = index_to_label[predicted_class]
    confidence = np.max(prediction)
    
    return class_label, confidence

# Main function
if __name__ == "__main__":
    # Paths to train and validation directories
    train_data_dir = 'C:/projects/SLT_project_MP_images/train'
    val_data_dir = 'C:/projects/SLT_project_MP_images/valid'

    # Generate label mappings
    classes = sorted(os.listdir(train_data_dir))  # Ensure consistent order
    label_to_index = {cls: idx for idx, cls in enumerate(classes)}
    index_to_label = {idx: cls for cls, idx in label_to_index.items()}
    print(label_to_index)
    print(index_to_label)
    # Build and train the model
    gesture_model = build_hand_gesture_classifier_lstm()
    gesture_model.summary()
    history = train_model(gesture_model, train_data_dir, val_data_dir, label_to_index, index_to_label)


{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [49]:
test_data_dir = "C:/projects/SLT_project_MP_images/test"
x_test, y_test = prepare_dataset(test_data_dir, label_to_index)

# Perform predictions
y_pred = np.argmax(gesture_model.predict(x_test, batch_size=16), axis=1)

# Calculate accuracy
test_accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Test Accuracy: 93.33%


In [ ]:
# You can load the cnn model it is the same perfromance
model_save_path = "C:/projects/SLT_project_MP_images/model.keras"
gesture_model.save(model_save_path)
# print("Model saved successfully!")


## Use the model

In [3]:
from tensorflow.keras.models import load_model
model_load_path = "C:/projects/SLT_project_MP_images/model.keras"

# Load the model
gesture_model = load_model(model_load_path)

In [4]:
# Extract hand landmarks function with drawing
def extract_hand_landmarks(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks and connections on the frame
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
            )

            # Extract landmarks
            landmark_list = []
            for lm in hand_landmarks.landmark:
                landmark_list.append(lm.x)
                landmark_list.append(lm.y)
            return np.array(landmark_list).flatten(), frame  # Return landmarks and updated frame
    
    # Return array of zeros and original frame if no hands are detected
    return np.zeros(42), frame
classes = sorted(os.listdir(train_data_dir))  # Ensure consistent order
label_to_index = {cls: idx for idx, cls in enumerate(classes)}
index_to_label = {idx: cls for cls, idx in label_to_index.items()}

In [5]:
def real_time_prediction_with_recording(model, index_to_label, output_video_path="recorded_output.avi"):
    cap = cv2.VideoCapture(0)  # Open the default camera

    # Set up video writer to save the recording
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for .avi files
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    print(f"Recording started! Video will be saved to: {output_video_path}")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Extract landmarks
        landmarks, frame_with_drawing = extract_hand_landmarks(frame)

        # Prepare data for prediction
        input_data = np.expand_dims(landmarks, axis=0)  # Shape (1, 42)

        # Predict gesture
        prediction = model.predict(input_data)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction)
        predicted_label = index_to_label[predicted_class]  # Map predicted class to label

        # Display results on the frame
        cv2.putText(frame_with_drawing, f'Prediction: {predicted_label} ({confidence:.2f})', 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow('Real-Time Gesture Recognition', frame_with_drawing)

        # Write the frame to the video file
        out.write(frame_with_drawing)

        # Break the loop with 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Recording finished! Video saved to: {output_video_path}")

# Call the real-time prediction function with recording
real_time_prediction_with_recording(gesture_model, index_to_label, output_video_path="gesture_recognition.avi")


Recording started! Video will be saved to: gesture_recognition.avi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━